[Gradient Descent](https://www.kaggle.com/code/jhoward/how-does-a-neural-net-really-work/notebook)

In [ ]:
from fastai.basics import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

plt.rc('figure', dpi=90)
print('Matplotlib version:', mpl.__version__)
print('Has mpl.colors:', hasattr(mpl,'colors'))

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylimit=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylimit: 
        plt.ylim(ylimit)
    plt.plot(x, f(x), color)
    if title is not None: 
        plt.title(title)

In [ ]:
def f(x): return 3*x**2 + 2*x + 1

plot_function(f=f, title="$3x^2 + 2x + 1$")

In [ ]:
def quad(a, b, c, x): return a*x**2 + b*x + c

In [ ]:
def mk_quad(a,b,c): return partial(quad, a,b,c)

In [ ]:
f2 = mk_quad(3,2,1)
plot_function(f2)

In [ ]:
def noise(x, scale): 
    return np.random.normal(scale=scale, size=x.shape)

def add_noise(x, mult, add): 
    return x * (1+noise(x,mult)) + noise(x,add)

In [ ]:
np.random.seed(42)

x = torch.linspace(start=-2, end=2, steps=20)[:,None]
y = add_noise(f(x), 0.15, 1.5)

In [ ]:
x[:5], y[:5]

In [ ]:
plt.scatter(x,y);

In [ ]:
from ipywidgets import interact

@interact(
    a=(0.0,5.0,0.1),
    b=(0.0,5.0,0.1),
    c=(0.0,5.0,0.1)
)
def plot_quad(a=1.1, b=1.1, c=1.1):
    plt.scatter(x,y)
    plot_function(mk_quad(a,b,c), ylimit=(-3,13))

In [ ]:
def mae(preds, acts): 
    return (torch.abs(preds-acts)).mean()

In [ ]:
@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a, b, c):
    f = mk_quad(a,b,c)
    plt.scatter(x,y)
    loss = mae(f(x), y)
    plot_function(f, ylim=(-3,12), title=f"MAE: {loss:.2f}")

## Automating gradient descent

In [ ]:
def quad_mae(params):
    f = mk_quad(*params)
    return mae(f(x), y)

In [ ]:
quad_mae([1.1, 1.1, 1.1])

In [ ]:
abc = torch.tensor([1.1,1.1,1.1])

In [ ]:
abc.requires_grad_()

In [ ]:
loss = quad_mae(abc)
loss

In [ ]:
loss.backward()

In [ ]:
abc.grad

In [ ]:
with torch.no_grad():
    abc -= abc.grad*0.01
    loss = quad_mae(abc)
    
print(f'loss={loss:.2f}')

In [ ]:
for i in range(10):
    loss = quad_mae(abc)
    loss.backward()
    with torch.no_grad(): abc -= abc.grad*0.01
    print(f'step={i}; loss={loss:.2f}')